In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
!ls '/content/drive/MyDrive/Colab_Notebooks/DL_final/'


BuzzFeed	       feature_matrix_pf.csv		  UserFeature.mat
BuzzFeedNewsUser.txt   news_news_bf_adjacency_matrix.csv  User.txt
BuzzFeedUserUser.txt   news_news_pf_adjacency_matrix.csv
feature_matrix_bf.csv  News.txt


In [4]:
!pwd

/content


In [5]:
import numpy as np
import tensorflow as tf
import argparse
import scipy.sparse as sp
import pandas as pd 
import random
import json
import os

from feature_matrix import FeatureMatrix

import pandas as pd 
import tensorflow_hub as hub
from bert import run_classifier
from bert import tokenization


In [6]:
FM = FeatureMatrix(base_path = "/content/drive/MyDrive/Colab_Notebooks/DL_final//")
label_zip = None

In [7]:
 def getAdj( dataset="BuzzFeed"):
        if dataset == "BuzzFeed":
            adj_np = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/DL_final/news_news_bf_adjacency_matrix.csv", header=None).values
        else:
            adj_np = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/DL_final/news_news_pf_adjacency_matrix.csv", header=None).values

        return sp.csr_matrix(adj_np, dtype=int)


In [8]:
    def getFeatures( dataset="BuzzFeed"):
        feature_df =  FM.get_feature_matrix(dataset)
        label = feature_df['label'].tolist()
        label_comp = [0 if each else 1 for each in label]
        global label_zip
        label_zip = list(zip(label_comp, label))
        feature_df.drop(['label'], axis=1)
        feature_np = feature_df.values

        return sp.csr_matrix(feature_np, dtype=float).tolil()



In [9]:
    def getYs(  dataset="BuzzFeed"):
        if dataset == "BuzzFeed":
            random.seed(1)
        else:
            random.seed(1)
        yTrain =  label_zip[:]
        yVal =  label_zip[:]
        yTest =  label_zip[:]
        train_mask = [False] * len(yTrain)
        val_mask = [False] * len(yTrain)
        test_mask = [False] * len(yTrain)
        n = len(yTrain)

        set_of_records_range = set(range(n))

        train_range = set(random.sample(set_of_records_range, k=int(n * 0.6)))
        set_of_records_range = set_of_records_range - train_range

        val_range = set(random.sample(set_of_records_range, k=int(n * 0.2)))
        set_of_records_range = set_of_records_range - train_range

        test_range = set(random.sample(set_of_records_range, k=int(n * 0.2)))

        for i in train_range:
            yVal[i] = (0,0)
            yTest[i] = (0,0)
            train_mask[i] = True
        for i in val_range:
            yTrain[i] = (0,0)
            yTest[i] = (0,0)
            val_mask[i] = True
        for i in test_range:
            yVal[i] = (0,0)
            yTrain[i] = (0,0)
            test_mask[i] = True

        return yTrain, yVal, yTest, train_mask, val_mask, test_mask


In [10]:
    def getComps(  dataset="BuzzFeed"):
        print(dataset)
        adj =  getAdj(dataset)
        features =  getFeatures(dataset)

        yTrain, yVal, yTest, train_mask, val_mask, test_mask =  getYs(dataset)

        return adj, features, yTrain, yVal, yTest, train_mask, val_mask, test_mask


In [11]:
def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape

    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx


In [12]:
def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1, dtype=float).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features.todense(), sparse_to_tuple(features)

In [13]:
def adj_to_bias(adj, sizes, nhood=1):
    nb_graphs = adj.shape[0]
    mt = np.empty(adj.shape)
    for g in range(nb_graphs):
        mt[g] = np.eye(adj.shape[1])
        for _ in range(nhood):
            mt[g] = np.matmul(mt[g], (adj[g] + np.eye(adj.shape[1])))
        for i in range(sizes[g]):
            for j in range(sizes[g]):
                if mt[g][i][j] > 0.0:
                    mt[g][i][j] = 1.0
    return -1e9 * (1.0 - mt)


In [14]:

# adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = getComps(dataset='BuzzFeed')
# features, spars = preprocess_features(features)

In [15]:
from __future__ import division
from __future__ import print_function

import time
import tensorflow as tf 
from  utils import *
from models import GCN, MLP

# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', '', 'kernel')
# flags.DEFINE_string('dataset', 'cora', 'Dataset string.')  # 'cora', 'citeseer', 'pubmed'
# flags.DEFINE_string('model', 'gcn', 'Model string.')  # 'gcn', 'gcn_cheby', 'dense'
# flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 16, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 10, 'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

# Load data
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = getComps(dataset='BuzzFeed')
features=sp.csr_matrix(features)
# Some preprocessing
features = preprocess_features(features)
# if FLAGS.model == 'gcn':
support = [preprocess_adj(adj)]
num_supports = 1
model_func = GCN
# elif FLAGS.model == 'gcn_cheby':
#     support = chebyshev_polynomials(adj, FLAGS.max_degree)
#     num_supports = 1 + FLAGS.max_degree
#     model_func = GCN
# elif FLAGS.model == 'dense':
#     support = [preprocess_adj(adj)]  # Not used
#     num_supports = 1
#     model_func = MLP
# else:
#     raise ValueError('Invalid argument for model: ' + str(FLAGS.model))

y_train  = np.asarray(y_train)
y_val = np.asarray(y_val)
y_test = np.asarray(y_test)
train_mask  = np.asarray(train_mask)
val_mask  = np.asarray(val_mask)
test_mask  = np.asarray(test_mask)

# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}


BuzzFeed
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 182


INFO:tensorflow:Writing example 0 of 182


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)
/content/utils.py:126: RuntimeWarning: invalid value encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5).flatten()


In [16]:
    print('adj shape', type(adj))
    print('features shape', type(features))
    print('y_train shape', type(y_train))
    print('y_val shape', type(y_val))
    print('y_test shape', type(y_test))
    print('train_mask shape', type(train_mask))
    print('val_mask shape', type(val_mask))
    print('test_mask shape', type(test_mask)) 

adj shape <class 'scipy.sparse.csr.csr_matrix'>
features shape <class 'tuple'>
y_train shape <class 'numpy.ndarray'>
y_val shape <class 'numpy.ndarray'>
y_test shape <class 'numpy.ndarray'>
train_mask shape <class 'numpy.ndarray'>
val_mask shape <class 'numpy.ndarray'>
test_mask shape <class 'numpy.ndarray'>


In [17]:
train_mask.shape

(182,)

In [ ]:
adj shape <class 'scipy.sparse.csr.csr_matrix'>
features shape <class 'scipy.sparse.lil.lil_matrix'>
y_train shape <class 'numpy.ndarray'>
y_val shape <class 'numpy.ndarray'>
y_test shape <class 'numpy.ndarray'>
train_mask shape <class 'numpy.ndarray'>
val_mask shape <class 'numpy.ndarray'>
test_mask shape <class 'numpy.ndarray'>

In [18]:

# Create model
model = model_func(placeholders, input_dim=features[2][1], logging=True)

# Initialize session
sess = tf.Session()


# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], (time.time() - t_test)


# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []

# Train model
for epoch in range(FLAGS.epochs):

    t = time.time()
    # Construct feed dictionary
    print('before feed')
    feed_dict = construct_feed_dict(features, support, y_train, train_mask, placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Validation
    cost, acc, duration = evaluate(features, support, y_val, val_mask, placeholders)
    cost_val.append(cost)

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and cost_val[-1] > np.mean(cost_val[-(FLAGS.early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

# Testing
test_cost, test_acc, test_duration = evaluate(features, support, y_test, test_mask, placeholders)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



before feed
Epoch: 0001 train_loss= nan train_acc= 0.48624 val_loss= nan val_acc= 0.83333 time= 0.30413
before feed
Epoch: 0002 train_loss= nan train_acc= 0.48624 val_loss= nan val_acc= 0.83333 time= 0.01034
before feed
Epoch: 0003 train_loss= nan train_acc= 0.48624 val_loss= nan val_acc= 0.83333 time= 0.01057
before feed
Epoch: 0004 train_loss= nan train_acc= 0.48624 val_loss= nan val_acc= 0.83333 time= 0.01009
before feed
Epoch: 0005 train_loss= nan train_acc= 0.48624 val_loss= nan val_acc= 0.83333 time= 0.01023
before feed
Epoch: 0006 train_loss= nan train_acc= 0.48624 val_loss= nan val_acc= 0.83333 time= 0.01063
before feed
Epoch: 0007 train_loss= nan train_acc= 0.48624 val_loss= nan val_acc= 0.83333 time= 0.00961
before feed
Epoch: 0008 train_loss= nan train_acc= 0.48624 val_loss= nan val_acc= 0.83333 time= 0.00915
before feed
Epoch: 0009 train_loss= nan train_acc= 0.48624 val_loss= nan val_acc= 0.83333 time= 0.00991
before feed
Epoch: 0010 train_loss= nan train_acc= 0.48624 val_l

In [19]:
feed_dict

{<tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f34688da630>: (array([[  0,   0],
         [  7,   0],
         [ 23,   0],
         ...,
         [178, 181],
         [180, 181],
         [181, 181]], dtype=int32),
  array([0.04651163, 0.01921301, 0.03327792, ...,        nan,        nan,
                nan]),
  (182, 182)),
 <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f3403e64588>: (array([[  0, 127],
         [  0, 126],
         [  0, 125],
         ...,
         [181,   2],
         [181,   1],
         [181,   0]], dtype=int32),
  array([0.00020374, 0.00404288, 0.0040249 , ..., 0.00597616, 0.0135185 ,
         0.00023126]),
  (182, 129)),
 <tf.Tensor 'PlaceholderWithDefault:0' shape=() dtype=float32>: 0.5,
 <tf.Tensor 'Placeholder_5:0' shape=(?, 2) dtype=float32>: array([[1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 0],
        [0, 0],
        [0, 0],
        [1,